In [ ]:
from groq import Groq
from openai import OpenAI
import os
import json
import time
import tqdm

groq_client = Groq(
    api_key=groq_api_key,
)


temperature = 0.0
    
# # groq inference
# def inference_groq(content, model):
#     chat_completion = groq_client.chat.completions.create(
#             messages=[
#                 {"role": "system", "content": """
#                 You are an expert at transliterating Greeklish into Greek. Transliterate the user's input from Greeklish into Greek, while following these rules:
#                 - Make sure you output the transliterated text only
#                 - Do not correct any mistakes of the user and do not change the spacing of the words
#                 - If a word is not in greek, leave it as it is
#                 - Do not translate english words into greek, leave them as is """},
#                 {
#                     "role": "user",
#                     "content": content
#                 }
#             ],
#             model=model,
#             temperature=temperature,
#         )
#     return chat_completion.choices[0].message.content


# openAI inference
def inference_LLM(content, model, client):
    chat_completion = client.chat.completions.create(
        temperature=temperature,
        model=model,
        messages=[
                {"role": "system", "content": """
                You are an expert at transliterating Greeklish into Greek. Transliterate the user's input from Greeklish into Greek, while following these rules:
                - Make sure you output the transliterated text only
                - Do not correct any mistakes of the user and do not change the spacing of the words
                - If a word is not in greek, leave it as it is
                - Do not translate english words into greek, leave them as is """},
                {
                    "role": "user",
                    "content": content
                }
            ]
        )
    return chat_completion.choices[0].message.content

In [20]:
# response = inference_groq("ti na kanoume", "llama-3.1-70b-versatile")

In [18]:
# inference on the synthetic data
path = '/home/sp1r05/Documents/gsoc/greeklish_extended/mixed_greeklish_evaluation/evaluation_data/mixed_evaluation_data_300_sentences.json'
base_path = '/home/sp1r05/Documents/gsoc/greeklish_extended/mixed_greeklish_evaluation/evaluation_data/'


# model = "llama-3.1-70b-versatile"
# model = "gpt-4o-mini"
temperature = 0.0
models = ['llama-3.1-70b-versatile', 'llama-3.1-8b-instant', 'gemma2-9b-it', 'llama-3.2-3b-preview', 'llama-3.2-1b-preview', 'gpt-4o', 'gpt-4o-mini']

data_paths = os.listdir(base_path)

# Get the avaliable substitution percentages from the names of the files
# percentages = [float(name.split('_')[5]) for name in data_paths]


# with open(base_path, 'r') as f:
#     data = json.load(f)

# Iterate over the sentences and perform inference

models = ['llama-3.1-70b-versatile']

groq_client = Groq(
    api_key=groq_api_key,
)

openAI_client = OpenAI(api_key = openAI_key)

# Iterate over the models and perform inference
for model in models:
    print(model)
    if(model.split('-')[0] == 'gpt'):
        client = openAI_client
    else:
        client = groq_client

    # Iterate over the different files with varying degrees of substitution
    for data_path in data_paths:
        percentage = float(data_path.split('_')[5])
        if(percentage != 0.05):
            continue
        print(percentage)

        with open(os.path.join(base_path, data_path), 'r') as f:
            sentences = json.load(f)

        transliterated_data = []

        # Iterate over the sentences
        for i, sentence in tqdm.tqdm(enumerate(sentences)):
            transliterated_text = inference_LLM(sentence['greeklish'], model, client)
            
            transliterated_data.append({
                "greeklish": sentence['greeklish'],
                "greek": sentence['greek'],
                "mixed": sentence['mixed'],
                'greek_predicted': transliterated_text,
                'gt_indices': sentence['gt_indices']
            })

            if not os.path.exists(f"robustness_test/{model}"):
                os.makedirs(f"robustness_test/{model}")

            destination_path = f"robustness_test/{model}/{percentage}_synthetic_data_{len(sentences)}2.json"

            # Save the transliterated data
            with open(destination_path, "w") as f:
                json.dump(transliterated_data, f, ensure_ascii=False, indent=4)
            
            # time.sleep(0.2)
    

# for i, sentence in enumerate(data):
#     transliterated_text = inference_openAI(sentence['greeklish'], model)
    
#     transliterated_data.append({
#         "greeklish": sentence['greeklish'],
#         "greek": sentence['greek'],
#         "mixed": sentence['mixed'],
#         'greek_predicted': transliterated_text,
#         'gt_indices': sentence['gt_indices']
#     })
    
    

# folder_path = f"LLM_data/{model}_{temperature}_synthetic_data"

# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)

#     # Save the transliterated data
# with open(f"{folder_path}/synthetic_data_{len(data)}.json", "w") as f:
#     json.dump(transliterated_data, f, ensure_ascii=False, indent=4)

llama-3.1-70b-versatile
0.05


35it [00:35,  1.02s/it]


KeyboardInterrupt: 